In [16]:
import pandas as pd
from glob import glob
from sklearn.metrics import roc_auc_score
from analysis.main_analyzer import MainAnalyzer
from analysis.utils import drop_weird_rows
from catboost import CatBoostClassifier, Pool,CatBoost, CatBoostRegressor


In [17]:
questions = pd.read_csv("vkcup/data.csv", sep=";", index_col="ID")
test = pd.read_csv("vkcup/test.csv", sep=";", index_col="ID")
test['Answer'] = 0
train = pd.read_csv("vkcup/train.csv", sep=";", index_col="ID")
data = questions.join(train)
drop_weird_rows(data)
# data=data[data.index<30000]

In [18]:
main_analyzer = MainAnalyzer(data)
main_analyzer(data)


In [19]:
train=data[data.index<=30000]
val=data[(data.index>30000)] 
# add_train = val[val.sure_is_bad==1]
# add_train.Answer = 0
# train = pd.concat([train,add_train])

In [20]:
for df in [train, val]:
    for c in ['count_NET',
              'has_start_brackets_type3','is_balanced_brackets_type2', 'sure_is_bad', 'Question',
             'count_start_quote_type5', 'is_balanced_brackets_type3', 'count_start_quote_type3',
              'has_end_brackets_type3'
             ]:
        del df[c]


In [21]:
# train=train[~train.sure_is_bad]

Y = train.Answer
del train['Answer']
X = train

Y_val = val.Answer
del val['Answer']
X_val = val

In [22]:

cat_features=['most_popular_word', 'first_popular_word', 'first_word', 'second_word', 'last_word']

model = CatBoostClassifier(iterations=4000, 
                           scale_pos_weight=1/Y.mean(),
                           learning_rate=0.01,
                           task_type="GPU",
                           eval_metric='AUC',
                           cat_features=cat_features,
                          )
model.fit(X,
          Y,
#           eval_set=(X_val, Y_val),
          verbose=True)

# 999:	learn: 0.8756470	test: 0.8404123	best: 0.8409981 (699)	total: 36.6s	remaining: 0us
# 0.0.8411099

0:	learn: 0.8112338	total: 53.3ms	remaining: 3m 33s
1:	learn: 0.8296536	total: 107ms	remaining: 3m 34s
2:	learn: 0.8297965	total: 162ms	remaining: 3m 36s
3:	learn: 0.8343687	total: 198ms	remaining: 3m 18s
4:	learn: 0.8336018	total: 236ms	remaining: 3m 8s
5:	learn: 0.8335852	total: 275ms	remaining: 3m 3s
6:	learn: 0.8340250	total: 311ms	remaining: 2m 57s
7:	learn: 0.8335834	total: 346ms	remaining: 2m 52s
8:	learn: 0.8342086	total: 381ms	remaining: 2m 49s
9:	learn: 0.8356853	total: 418ms	remaining: 2m 46s
10:	learn: 0.8356622	total: 454ms	remaining: 2m 44s
11:	learn: 0.8355726	total: 494ms	remaining: 2m 44s
12:	learn: 0.8378273	total: 533ms	remaining: 2m 43s
13:	learn: 0.8379582	total: 569ms	remaining: 2m 42s
14:	learn: 0.8373667	total: 606ms	remaining: 2m 40s
15:	learn: 0.8373771	total: 640ms	remaining: 2m 39s
16:	learn: 0.8395353	total: 676ms	remaining: 2m 38s
17:	learn: 0.8391605	total: 714ms	remaining: 2m 37s
18:	learn: 0.8394923	total: 750ms	remaining: 2m 37s
19:	learn: 0.8389696	to

159:	learn: 0.8534751	total: 6.61s	remaining: 2m 38s
160:	learn: 0.8535596	total: 6.65s	remaining: 2m 38s
161:	learn: 0.8536650	total: 6.69s	remaining: 2m 38s
162:	learn: 0.8537187	total: 6.74s	remaining: 2m 38s
163:	learn: 0.8537553	total: 6.78s	remaining: 2m 38s
164:	learn: 0.8538895	total: 6.82s	remaining: 2m 38s
165:	learn: 0.8539611	total: 6.86s	remaining: 2m 38s
166:	learn: 0.8540502	total: 6.9s	remaining: 2m 38s
167:	learn: 0.8541168	total: 6.94s	remaining: 2m 38s
168:	learn: 0.8541519	total: 6.97s	remaining: 2m 38s
169:	learn: 0.8542727	total: 7.01s	remaining: 2m 37s
170:	learn: 0.8543580	total: 7.05s	remaining: 2m 37s
171:	learn: 0.8543692	total: 7.09s	remaining: 2m 37s
172:	learn: 0.8544556	total: 7.12s	remaining: 2m 37s
173:	learn: 0.8545751	total: 7.16s	remaining: 2m 37s
174:	learn: 0.8545805	total: 7.2s	remaining: 2m 37s
175:	learn: 0.8546751	total: 7.23s	remaining: 2m 37s
176:	learn: 0.8547626	total: 7.27s	remaining: 2m 37s
177:	learn: 0.8548163	total: 7.31s	remaining: 2m

316:	learn: 0.8606868	total: 12.9s	remaining: 2m 29s
317:	learn: 0.8607135	total: 12.9s	remaining: 2m 29s
318:	learn: 0.8607293	total: 13s	remaining: 2m 29s
319:	learn: 0.8607431	total: 13s	remaining: 2m 29s
320:	learn: 0.8607784	total: 13.1s	remaining: 2m 29s
321:	learn: 0.8608143	total: 13.1s	remaining: 2m 29s
322:	learn: 0.8608739	total: 13.2s	remaining: 2m 29s
323:	learn: 0.8608947	total: 13.2s	remaining: 2m 29s
324:	learn: 0.8608999	total: 13.2s	remaining: 2m 29s
325:	learn: 0.8609247	total: 13.3s	remaining: 2m 29s
326:	learn: 0.8609608	total: 13.3s	remaining: 2m 29s
327:	learn: 0.8610017	total: 13.4s	remaining: 2m 29s
328:	learn: 0.8610112	total: 13.4s	remaining: 2m 29s
329:	learn: 0.8610479	total: 13.5s	remaining: 2m 29s
330:	learn: 0.8611040	total: 13.5s	remaining: 2m 29s
331:	learn: 0.8611339	total: 13.5s	remaining: 2m 29s
332:	learn: 0.8611451	total: 13.6s	remaining: 2m 29s
333:	learn: 0.8611644	total: 13.6s	remaining: 2m 29s
334:	learn: 0.8611825	total: 13.7s	remaining: 2m 2

473:	learn: 0.8645158	total: 19.2s	remaining: 2m 23s
474:	learn: 0.8645258	total: 19.3s	remaining: 2m 23s
475:	learn: 0.8645391	total: 19.3s	remaining: 2m 23s
476:	learn: 0.8645695	total: 19.4s	remaining: 2m 23s
477:	learn: 0.8646288	total: 19.4s	remaining: 2m 22s
478:	learn: 0.8646820	total: 19.4s	remaining: 2m 22s
479:	learn: 0.8647175	total: 19.5s	remaining: 2m 22s
480:	learn: 0.8647142	total: 19.5s	remaining: 2m 22s
481:	learn: 0.8647743	total: 19.6s	remaining: 2m 22s
482:	learn: 0.8648258	total: 19.6s	remaining: 2m 22s
483:	learn: 0.8648528	total: 19.6s	remaining: 2m 22s
484:	learn: 0.8648857	total: 19.7s	remaining: 2m 22s
485:	learn: 0.8649099	total: 19.7s	remaining: 2m 22s
486:	learn: 0.8649355	total: 19.8s	remaining: 2m 22s
487:	learn: 0.8649492	total: 19.8s	remaining: 2m 22s
488:	learn: 0.8649951	total: 19.9s	remaining: 2m 22s
489:	learn: 0.8650029	total: 19.9s	remaining: 2m 22s
490:	learn: 0.8650149	total: 19.9s	remaining: 2m 22s
491:	learn: 0.8650603	total: 20s	remaining: 2m

632:	learn: 0.8676114	total: 25.6s	remaining: 2m 16s
633:	learn: 0.8676179	total: 25.7s	remaining: 2m 16s
634:	learn: 0.8676240	total: 25.7s	remaining: 2m 16s
635:	learn: 0.8676327	total: 25.7s	remaining: 2m 16s
636:	learn: 0.8676420	total: 25.8s	remaining: 2m 15s
637:	learn: 0.8676738	total: 25.8s	remaining: 2m 15s
638:	learn: 0.8676946	total: 25.8s	remaining: 2m 15s
639:	learn: 0.8677049	total: 25.9s	remaining: 2m 15s
640:	learn: 0.8677368	total: 25.9s	remaining: 2m 15s
641:	learn: 0.8677612	total: 25.9s	remaining: 2m 15s
642:	learn: 0.8677685	total: 26s	remaining: 2m 15s
643:	learn: 0.8677933	total: 26s	remaining: 2m 15s
644:	learn: 0.8678069	total: 26.1s	remaining: 2m 15s
645:	learn: 0.8678203	total: 26.1s	remaining: 2m 15s
646:	learn: 0.8678205	total: 26.1s	remaining: 2m 15s
647:	learn: 0.8678274	total: 26.2s	remaining: 2m 15s
648:	learn: 0.8678496	total: 26.2s	remaining: 2m 15s
649:	learn: 0.8678617	total: 26.2s	remaining: 2m 15s
650:	learn: 0.8679069	total: 26.3s	remaining: 2m 1

...

3273:	learn: 0.8863008	total: 2m 10s	remaining: 28.9s
3274:	learn: 0.8863009	total: 2m 10s	remaining: 28.9s
3275:	learn: 0.8863009	total: 2m 10s	remaining: 28.8s
3276:	learn: 0.8863025	total: 2m 10s	remaining: 28.8s
3277:	learn: 0.8863026	total: 2m 10s	remaining: 28.8s
3278:	learn: 0.8863069	total: 2m 10s	remaining: 28.7s
3279:	learn: 0.8863076	total: 2m 10s	remaining: 28.7s
3280:	learn: 0.8863156	total: 2m 10s	remaining: 28.7s
3281:	learn: 0.8863201	total: 2m 10s	remaining: 28.6s
3282:	learn: 0.8863192	total: 2m 10s	remaining: 28.6s
3283:	learn: 0.8863205	total: 2m 10s	remaining: 28.5s
3284:	learn: 0.8863206	total: 2m 10s	remaining: 28.5s
3285:	learn: 0.8863420	total: 2m 10s	remaining: 28.5s
3286:	learn: 0.8863448	total: 2m 11s	remaining: 28.4s
3287:	learn: 0.8863484	total: 2m 11s	remaining: 28.4s
3288:	learn: 0.8863621	total: 2m 11s	remaining: 28.3s
3289:	learn: 0.8863641	total: 2m 11s	remaining: 28.3s
3290:	learn: 0.8863641	total: 2m 11s	remaining: 28.3s
3291:	learn: 0.8863641	total

3430:	learn: 0.8868054	total: 2m 16s	remaining: 22.7s
3431:	learn: 0.8868174	total: 2m 16s	remaining: 22.6s
3432:	learn: 0.8868190	total: 2m 16s	remaining: 22.6s
3433:	learn: 0.8868190	total: 2m 16s	remaining: 22.5s
3434:	learn: 0.8868190	total: 2m 16s	remaining: 22.5s
3435:	learn: 0.8868192	total: 2m 16s	remaining: 22.5s
3436:	learn: 0.8868192	total: 2m 16s	remaining: 22.4s
3437:	learn: 0.8868193	total: 2m 16s	remaining: 22.4s
3438:	learn: 0.8868193	total: 2m 16s	remaining: 22.3s
3439:	learn: 0.8868199	total: 2m 17s	remaining: 22.3s
3440:	learn: 0.8868277	total: 2m 17s	remaining: 22.3s
3441:	learn: 0.8868333	total: 2m 17s	remaining: 22.2s
3442:	learn: 0.8868333	total: 2m 17s	remaining: 22.2s
3443:	learn: 0.8868359	total: 2m 17s	remaining: 22.1s
3444:	learn: 0.8868370	total: 2m 17s	remaining: 22.1s
3445:	learn: 0.8868367	total: 2m 17s	remaining: 22.1s
3446:	learn: 0.8868366	total: 2m 17s	remaining: 22s
3447:	learn: 0.8868421	total: 2m 17s	remaining: 22s
3448:	learn: 0.8868453	total: 2m

3585:	learn: 0.8872338	total: 2m 23s	remaining: 16.5s
3586:	learn: 0.8872338	total: 2m 23s	remaining: 16.5s
3587:	learn: 0.8872338	total: 2m 23s	remaining: 16.5s
3588:	learn: 0.8872338	total: 2m 23s	remaining: 16.4s
3589:	learn: 0.8872419	total: 2m 23s	remaining: 16.4s
3590:	learn: 0.8872419	total: 2m 23s	remaining: 16.3s
3591:	learn: 0.8872422	total: 2m 23s	remaining: 16.3s
3592:	learn: 0.8872604	total: 2m 23s	remaining: 16.3s
3593:	learn: 0.8872604	total: 2m 23s	remaining: 16.2s
3594:	learn: 0.8872612	total: 2m 23s	remaining: 16.2s
3595:	learn: 0.8872611	total: 2m 23s	remaining: 16.1s
3596:	learn: 0.8872611	total: 2m 23s	remaining: 16.1s
3597:	learn: 0.8872696	total: 2m 23s	remaining: 16.1s
3598:	learn: 0.8872696	total: 2m 23s	remaining: 16s
3599:	learn: 0.8872781	total: 2m 23s	remaining: 16s
3600:	learn: 0.8872797	total: 2m 23s	remaining: 16s
3601:	learn: 0.8872828	total: 2m 24s	remaining: 15.9s
3602:	learn: 0.8872870	total: 2m 24s	remaining: 15.9s
3603:	learn: 0.8872870	total: 2m 2

3740:	learn: 0.8876174	total: 2m 30s	remaining: 10.4s
3741:	learn: 0.8876174	total: 2m 30s	remaining: 10.3s
3742:	learn: 0.8876176	total: 2m 30s	remaining: 10.3s
3743:	learn: 0.8876210	total: 2m 30s	remaining: 10.3s
3744:	learn: 0.8876211	total: 2m 30s	remaining: 10.2s
3745:	learn: 0.8876219	total: 2m 30s	remaining: 10.2s
3746:	learn: 0.8876317	total: 2m 30s	remaining: 10.1s
3747:	learn: 0.8876420	total: 2m 30s	remaining: 10.1s
3748:	learn: 0.8876593	total: 2m 30s	remaining: 10.1s
3749:	learn: 0.8876599	total: 2m 30s	remaining: 10s
3750:	learn: 0.8876626	total: 2m 30s	remaining: 9.99s
3751:	learn: 0.8876626	total: 2m 30s	remaining: 9.95s
3752:	learn: 0.8876643	total: 2m 30s	remaining: 9.91s
3753:	learn: 0.8876715	total: 2m 30s	remaining: 9.87s
3754:	learn: 0.8876794	total: 2m 30s	remaining: 9.83s
3755:	learn: 0.8876793	total: 2m 30s	remaining: 9.79s
3756:	learn: 0.8876820	total: 2m 30s	remaining: 9.75s
3757:	learn: 0.8876842	total: 2m 30s	remaining: 9.71s
3758:	learn: 0.8876846	total: 

3894:	learn: 0.8880698	total: 2m 36s	remaining: 4.21s
3895:	learn: 0.8880818	total: 2m 36s	remaining: 4.17s
3896:	learn: 0.8880824	total: 2m 36s	remaining: 4.13s
3897:	learn: 0.8880830	total: 2m 36s	remaining: 4.09s
3898:	learn: 0.8880830	total: 2m 36s	remaining: 4.05s
3899:	learn: 0.8880924	total: 2m 36s	remaining: 4.01s
3900:	learn: 0.8880919	total: 2m 36s	remaining: 3.97s
3901:	learn: 0.8880923	total: 2m 36s	remaining: 3.93s
3902:	learn: 0.8880924	total: 2m 36s	remaining: 3.89s
3903:	learn: 0.8880928	total: 2m 36s	remaining: 3.85s
3904:	learn: 0.8880944	total: 2m 36s	remaining: 3.81s
3905:	learn: 0.8881017	total: 2m 36s	remaining: 3.77s
3906:	learn: 0.8881017	total: 2m 36s	remaining: 3.73s
3907:	learn: 0.8881019	total: 2m 36s	remaining: 3.69s
3908:	learn: 0.8881136	total: 2m 36s	remaining: 3.65s
3909:	learn: 0.8881136	total: 2m 36s	remaining: 3.61s
3910:	learn: 0.8881135	total: 2m 36s	remaining: 3.57s
3911:	learn: 0.8881134	total: 2m 37s	remaining: 3.53s
3912:	learn: 0.8881224	total

In [23]:
model.get_feature_importance(prettified=True)[-10:]

,Feature Id,Importances
50,is_balanced_quote_type3,5.710224e-03
51,is_balanced_quote_type5,3.878005e-03
52,ends_with_question_sign_quote_type1,3.488465e-03
53,count_end_quote_type5,1.058709e-03
54,count_end_quote_type3,6.467173e-04
55,has_end_brackets_type4,6.044742e-05
56,has_end_brackets_type2,5.567610e-05
57,is_balanced_brackets_type4,4.601409e-05
58,has_start_brackets_type2,3.940868e-05
59,has_start_brackets_type4,1.846920e-07


In [25]:

val=data[data.index>30000]
main_analyzer(val)

In [26]:
val['predict'] = model.predict_proba(X_val)[:,1]


/home/ivan/projects/vkcup/venv/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [11]:
roc_auc_score(val['Answer'].values, val['predict'].values)

ValueError: Input contains NaN, infinity or a value too large for dtype('float64').

In [28]:
val[['predict']].to_csv('predicts2.csv',header=False)